In [ ]:
from tensorflow.python.tools import inspect_checkpoint as chkp
model_path = "/Users/claudecoulombe/git/tensorflow_basic_tutorial/model/"
chkp.print_tensors_in_checkpoint_file(model_path+'model.ckpt', tensor_name='', all_tensors=True)

In [8]:
# Testing model on fresh data, in fact synthesized data
# only for the purpose of getting an idea of the generalization of the model
import matplotlib.pyplot as plt 
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from tensorflow.python.framework import ops

# Create new data from the dataset "sonar.csv" modifying the features manually 
# creating 8 «synthetized» new data and save the file as new_sonar.csv
def read_dataset(file_name):
    dir_path = "/Users/claudecoulombe/git/tensorflow_basic_tutorial/"
    df = pd.read_csv(dir_path+file_name,delimiter=',')
    print("Nbr columns: ",len(df.columns))
    X = df[df.columns[0:60]].values
    y = df[df.columns[60]]
    # Encode the dependant variable
    encoder = LabelEncoder()
    encoder.fit(y)
    y = encoder.transform(y)
    Y = one_hot_encode(y)
    print("X.shape",X.shape)
    return (X,Y)
 
# Define the encoder function M => 1, R => 0
def one_hot_encode(labels):
    n_labels = len(labels)
#     n_unique_labels = len(np.unique(labels))
    n_unique_labels = 2
    one_hot_encode = np.zeros((n_labels,n_unique_labels))
    one_hot_encode[np.arange(n_labels),labels] = 1
    return one_hot_encode

# Read the dataset
X, Y = read_dataset('new_sonar.csv')

n_dim = X.shape[1] 
n_class = 2

# Define the number of hidden layers an the
# number of neurons for each layer
n_hidden_1 = 60
n_hidden_2 = 60
n_hidden_3 = 60
n_hidden_4 = 60

def multilayer_perceptron(x, weights, biases):
    # Hidden layer with RELU activations
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    # Hidden layer with sigmoid activations
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    layer_2 = tf.nn.sigmoid(layer_2)
    # Hidden layer with sigmoid activations
    layer_3 = tf.add(tf.matmul(layer_2, weights['h3']), biases['b3'])
    layer_3 = tf.nn.sigmoid(layer_3)
    # Hidden layer with RELU activations
    layer_4 = tf.add(tf.matmul(layer_3, weights['h4']), biases['b4'])
    layer_4 = tf.nn.relu(layer_4)
    # Output layer with linear activations
    out_layer = tf.matmul(layer_4, weights['out']) + biases['out_b']
    return out_layer
 
tf.reset_default_graph() 

model_path = "/Users/claudecoulombe/git/tensorflow_basic_tutorial/model/"
new_saver = tf.train.import_meta_graph(model_path+'model.ckpt.meta')

with tf.Session() as sess:  
    new_saver.restore(sess, tf.train.latest_checkpoint(model_path))

    # Inputs and outputs
    x = tf.placeholder(tf.float32,[None, n_dim])
    y_ = tf.placeholder(tf.float32,[None, n_class])

    # define the weights and the biases for each layer
    weights = {
        'h1': sess.run("h1:0"),
        'h2': sess.run("h2:0"),
        'h3': sess.run("h3:0"),
        'h4': sess.run("h4:0"),
        'out': sess.run("out:0"),
        }
    biases = {
        'b1': sess.run("b1:0"),
        'b2': sess.run("b2:0"),
        'b3': sess.run("b3:0"),
        'b4': sess.run("b4:0"),
        'out_b': sess.run("out_b:0"),
        }

    # Call your model defined
   
    print("***************** Prediction on testing data *****************")
    print("**************************************************************")
    print("* 1 stands for M, (i.e. Mine) and 0 stands for R (i.e. Rock) *")
    print("**************************************************************")
    total_success = 0
    total_new_data_samples = 8
    for i in range(total_new_data_samples):
        # Call your model defined
        prediction_run = sess.run(tf.argmax(multilayer_perceptron(x, weights, biases), 1), feed_dict={x:X[i].reshape(1,60)})
        accuracy_run = sess.run(tf.reduce_mean(tf.cast(tf.equal(tf.argmax(multilayer_perceptron(x, weights, biases), 1), tf.argmax(y_,1)), tf.float32)), feed_dict={x:X[i].reshape(1,60), y_:Y[i].reshape(1,2)})
        print(i,"Original Class: ", int(sess.run(y_[i][1],feed_dict={y_:Y})), " Predicted Values: ", prediction_run[0] )
        print("Accuracy: ",str(accuracy_run*100)+"%")
        if accuracy_run == 1:
            total_success += 1 
    print("Final accuracy:",str(total_success/total_new_data_samples*100)+"%")

Nbr columns:  61
X.shape (8, 60)
INFO:tensorflow:Restoring parameters from /Users/claudecoulombe/git/tensorflow_basic_tutorial/model/model.ckpt
***************** Prediction on testing data *****************
**************************************************************
* 1 stands for M, (i.e. Mine) and 0 stands for R (i.e. Rock) *
**************************************************************
0 Original Class:  0  Predicted Values:  0
Accuracy:  100.0%
1 Original Class:  0  Predicted Values:  0
Accuracy:  100.0%
2 Original Class:  0  Predicted Values:  0
Accuracy:  100.0%
3 Original Class:  1  Predicted Values:  0
Accuracy:  0.0%
4 Original Class:  0  Predicted Values:  0
Accuracy:  100.0%
5 Original Class:  0  Predicted Values:  0
Accuracy:  100.0%
6 Original Class:  0  Predicted Values:  0
Accuracy:  100.0%
7 Original Class:  0  Predicted Values:  0
Accuracy:  100.0%
Final accuracy: 87.5%


With 87.5% , the model has shown some power of generalization.